## Class Praproses Data

In [1]:
# pustaka manipulasi data array
import numpy as np
from numpy import concatenate

# pustaka manipulasi data frame
import pandas as pd
from pandas import concat
from pandas import DataFrame

# pustaka tanggal
from datetime import datetime

# pustaka normalisasi data
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from mlxtend.preprocessing import minmax_scaling

# pustaka data latih dan data uji
from sklearn.model_selection import train_test_split

- fungsi aggregasi data

In [2]:
## => Aggregasi data harian
def hotspot_harian(df):
    # Menghitung jumlah titik panas berdasarkan tanggal
    hasil = df.groupby(['acq_date']).size().reset_index(name='hotspot')

    # memalukan indexing berdasarkan tanggal
    hasil = hasil.set_index("acq_date")
    
    # return value data hotspot harian
    return hasil

In [3]:
## => Aggregasi data bulanan
def hotspot_bulanan(df):
    # Menghitung jumlah titik panas berdasarkan tanggal
    hasil = df.groupby(['acq_date']).size().reset_index(name='hotspot')

    # memalukan indexing berdasarkan tanggal
    hasil = hasil.set_index("acq_date")

    # proses aggregasi ke data bulanan
    hasil = hasil.resample('M').sum()

    # return value data hotspot harian
    return hasil

In [4]:
## => Aggregasi data bulanan
def hotspot_tahunan(df):
    # Menghitung jumlah titik panas berdasarkan tanggal
    hasil = df.groupby(['acq_date']).size().reset_index(name='hotspot')

    # memalukan indexing berdasarkan tanggal
    hasil = hasil.set_index("acq_date")

    # proses aggregasi ke data bulanan
    hasil = hasil.resample('Y').sum()

    # return value data hotspot harian
    return hasil

- fungsi normalisasi

In [5]:
# membuat fungsi distribusi normal
# menggunakan pustaka scikit-learn
def normalisasi(df):
    hasil = preprocessing.normalize(df)
    return hasil

In [6]:
# membuat fungsi z-score scaler
# menggunakan pustaka scikit-learn
# def normalisasi_z(df, nm_kolom): # belum selesai
    # return hasil

In [7]:
# membuat fungsi max-min scaler
# menggunakan pustaka scikit-learn
def normalisasi_max_min(df):
    # memanggil fungsi max min scaler
    hasil = preprocessing.MinMaxScaler(feature_range=(-1, 1))
    
    # proses max min scaler
    hasil = hasil.fit_transform(df)
    
    # pengembalian nilai
    return hasil

In [8]:
# membuat fungsi max-min scaler
# menggunakan pustaka mlxtend
def scaling_max_min(df, nm_kolom):
    hasil = minmax_scaling(df, columns=nm_kolom)
    return hasil

- fungsi supervised learning

In [9]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    
    # return value
    return agg